<a href="https://colab.research.google.com/github/rebnoob/PressDatabase/blob/main/Member_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Prerequisite
Just run it

In [96]:
!pip install gnews

#Get News from Google

## Setup

This code for importing the excel sheet. Make sure the excel sheet follows the format of the sample. Drag it to the file folder and change the name below to the name of the excel sheet. File name example "region1" or "region2". No CAP

In [102]:
import pandas as pd

filename = "nameList.xlsx" #@param {type:"string"}
sheet_name = "Region5" #@param {type:"string"}

df = pd.read_excel(filename, sheet_name=sheet_name)
df


,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Casar,Gregorio,Eduardo,NaN,NaN,Greg Casar,1989-05-04,M,rep,TX,...,NaN,NaN,H2TX35144,NaN,456945,161953.0,NaN,NaN,NaN,Greg Casar
1,Babin,Brian,NaN,NaN,NaN,Brian Babin,1948-03-23,M,rep,TX,...,N00005736,NaN,H6TX02079,44883.0,412655,360.0,Brian Babin,NaN,21551.0,Brian Babin
2,Doggett,Lloyd,A.,NaN,NaN,Lloyd Doggett,1946-10-06,M,rep,TX,...,N00006023,NaN,H4TX10028,36810.0,400111,21689.0,Lloyd Doggett,NaN,29571.0,Lloyd Doggett
3,Hunt,Wesley,Parish,NaN,NaN,Wesley Hunt,1981-11-13,M,rep,TX,...,NaN,NaN,H0TX07170,NaN,456946,188147.0,Wesley Hunt (Texas Congress),NaN,NaN,Wesley Hunt
4,Moore,Blake,David,NaN,NaN,Blake D. Moore,1980-06-22,M,rep,UT,...,N00046598,NaN,H0UT01205,NaN,456851,NaN,NaN,NaN,NaN,Blake Moore
5,Stewart,Chris,NaN,NaN,NaN,Chris Stewart,1960-07-15,M,rep,UT,...,N00033932,NaN,H2UT02324,68466.0,412581,135930.0,Chris Stewart,NaN,21367.0,Chris Stewart (politician)
6,Curtis,John,R.,NaN,NaN,John R. Curtis,1960-05-10,M,rep,UT,...,N00041221,NaN,H8UT03238,NaN,412740,123390.0,John Curtis (Utah),NaN,21755.0,John Curtis (Utah politician)
7,Owens,Clarence,NaN,NaN,Burgess,Burgess Owens,1951-08-02,M,rep,UT,...,N00045812,NaN,H0UT04076,NaN,456852,NaN,Burgess Owens,NaN,NaN,Burgess Owens
8,Wittman,Robert,J.,NaN,NaN,Robert J. Wittman,1959-02-03,M,rep,VA,...,N00029459,NaN,H8VA01147,1028089.0,412255,58133.0,Robert J. Wittman,NaN,20756.0,Rob Wittman
9,Kiggans,Jennifer,Ann,NaN,NaN,Jennifer Kiggans,1971-06-18,F,rep,VA,...,NaN,NaN,H2VA02064,NaN,456947,186387.0,NaN,NaN,NaN,Jen Kiggans


The start date doesn't work, just ignore

In [ ]:
google_news = GNews()
google_news.max_results = 10  # number of responses across a keyword
google_news.country = 'United States'  # News from a specific country
google_news.language = 'english'  # News in a specific language
google_news.exclude_websites = ["congress.gov",'house.gov',"senate.gov"]  # Exclude news from specific websites
google_news.start_date = (2023, 7, 1) # Search from 1st Jan 2020

In [ ]:
news_list = pd.DataFrame(columns=['Name','Title','Date','URL'])

## Get Individual

In [103]:
import re
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
from gnews import GNews

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def process_news(info, seven_days_ago, rep):
    date_format = '%a, %d %b %Y %H:%M:%S %Z'
    news_row = None
    try:
        async with aiohttp.ClientSession() as session:
            response = await fetch(session, info["url"])
            if datetime.strptime(info["published date"], date_format) >= seven_days_ago:
                if response and not response.startswith("https://consent.google.com/"):
                    #print(info["title"] + info["published date"])
                    final_url = extract_final_url(response)
                    if final_url:
                        #print(final_url)
                        news_row = {
                            'Name': rep,
                            'Title': info["title"],
                            'Date': info["published date"],
                            'URL': final_url
                        }
    except aiohttp.ClientError as e:
        print("URL not working:", e)

    return pd.DataFrame(news_row, index=[0]) if news_row else None  # Return None if conditions are not met

def extract_final_url(html_content):
    match = re.search(r'<a\s+href=[\'"]?([^\'" >]+)', html_content)
    if match:
        return match.group(1)
    return None


async def main():
    rep = "Rep.Mike Levin"
    google_news = GNews()
    google_news.max_results = 10
    google_news.country = 'United States'
    google_news.language = 'english'
    google_news.exclude_websites = ["congress.gov", 'house.gov', "senate.gov"]
    google_news.start_date = (2023, 7, 1)
    news_list = pd.DataFrame(columns=['Name', 'Title', 'Date', 'URL'])
    seven_days_ago = datetime.now() - timedelta(days=7)

    news = google_news.get_news(rep)

    tasks = []
    for info in news:
        task = asyncio.create_task(process_news(info, seven_days_ago, rep))
        tasks.append(task)

    result = await asyncio.gather(*tasks)
    news_list = pd.concat([df for df in result if df is not None], ignore_index=True)  # Concatenate non-empty DataFrames

    return news_list



In [104]:
if __name__ == '__main__':
    news_list = await main()

ValueError: ignored

In [ ]:
news_list

,Name,Title,Date,URL
0,Rep.Mike Levin,"In Orange County’s open congressional race, do...","Mon, 03 Jul 2023 16:30:37 GMT",https://www.ocregister.com/2023/07/03/in-orang...
1,Rep.Mike Levin,"Reps. Peters, Vargas, Jacobs, & Levin Submit N...","Fri, 30 Jun 2023 22:53:38 GMT",https://scottpeters.house.gov/2023/6/reps-pete...
2,Rep.Mike Levin,"Huffman, Levin to introduce bills to protect P...","Wed, 28 Jun 2023 03:30:59 GMT",https://www.triplicate.com/news/huffman-levin-...
3,Rep.Mike Levin,"Legislation Would Improve GI Bill, Other VA Se...","Tue, 27 Jun 2023 12:50:00 GMT",https://www.ngaus.org/newsroom/legislation-wou...


## List + Get Article

Once you load the documents, this should be fully automatic. It will find all the news about the rep for you. If this is taking too long to load, click on the loading button "ONCE" and only once, each time it gets stucked.

In [ ]:
import re
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
from gnews import GNews
from bs4 import BeautifulSoup
import time

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def process_news(info, seven_days_ago, rep):
    date_format = '%a, %d %b %Y %H:%M:%S %Z'
    news_row = None
    try:
        async with aiohttp.ClientSession() as session:
            response = await fetch(session, info["url"])
            # Check if the published date is within the last seven days
            if datetime.strptime(info["published date"], date_format) >= seven_days_ago:
                if response and not response.startswith("https://consent.google.com/"):
                    final_url = extract_final_url(response)
                    if final_url:
                        # Create a dictionary with the news information
                        news_row = {
                            'Name': rep,
                            'Title': info["title"],
                            'Date': info["published date"],
                            'URL': final_url
                        }
    except aiohttp.ClientError as e:
        print("URL not working:", e)

    # Return a DataFrame with the news information or None if no news row was created
    return pd.DataFrame(news_row, index=[0]) if news_row else None

def extract_final_url(html_content):
    # Use regex to extract the final URL from the HTML content
    match = re.search(r'<a\s+href=[\'"]?([^\'" >]+)', html_content)
    if match:
        return match.group(1)
    return None

async def fetch_result_text(index, url):
    lexper_url = "https://lexper.p.rapidapi.com/v1.1/extract"
    querystring = {"url": url, "js_timeout": "30", "media": "true"}
    headers = {
        "X-RapidAPI-Key": "8a62fe820amsh2a4a72771032a6cp1b6f99jsn339a1fcd31ca",
        "X-RapidAPI-Host": "lexper.p.rapidapi.com"
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(lexper_url, headers=headers, params=querystring) as response:
            data = await response.json()
            html = data.get('article', {}).get('html', '')
            htmlParse = BeautifulSoup(html, 'html.parser')
            text = ""
            for para in htmlParse.find_all(['p', 'h1', 'h2', 'h3']):
                text += para.get_text().lstrip() + "\n"
            return index, text

async def main(csv_filename):
    google_news = GNews()
    google_news.max_results = 10
    google_news.country = 'United States'
    google_news.language = 'english'
    google_news.exclude_websites = ["congress.gov", 'house.gov', "senate.gov"]
    news_list = pd.DataFrame(columns=['Name', 'Title', 'Date', 'URL'])
    seven_days_ago = datetime.now() - timedelta(days=7)

    # Read the CSV file and extract the 'full_name' column as a list of representatives
    representatives = pd.read_csv(csv_filename)['full_name'].tolist()

    tasks = []
    for rep in representatives:
        rep_name = "Rep." + rep
        # Get news articles for each representative
        news = google_news.get_news(rep_name)
        for info in news:
            # Create a task for processing each news article asynchronously
            task = asyncio.create_task(process_news(info, seven_days_ago, rep))
            tasks.append(task)

    # Gather all the results from the tasks
    result = await asyncio.gather(*tasks)

    # Concatenate the non-empty DataFrames into a single DataFrame
    news_list = pd.concat([df for df in result if df is not None], ignore_index=True)

    # Fetch the result text for each URL
    tasks = []
    result_texts = {}
    for i, url in enumerate(news_list['URL']):
        task = asyncio.create_task(fetch_result_text(i, url))
        tasks.append(task)
        await asyncio.sleep(1)  # Add a delay of 1 second between each request

    result_texts = await asyncio.gather(*tasks)

    # Update the 'ResultText' column in the DataFrame
    for index, text in result_texts:
        news_list.at[index, 'ResultText'] = text

    # Return the final DataFrame with news information
    return news_list



In [ ]:
if __name__ == '__main__':
# Run the main function
  csv_filename = 'Test2.csv'  # Replace with the actual CSV file name
  result = await main(csv_filename)

In [ ]:
result

# Get list without article text

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def get_news_articles(keyword, days):
    # Encode the keyword to be used in the URL
    encoded_keyword = keyword.replace(' ', '+')

    # Calculate the start date 'days' ago from the current date
    start_date = datetime.now() - timedelta(days=days)

    # Format the start date and current date
    formatted_start_date = start_date.strftime('%Y-%m-%d')
    formatted_current_date = datetime.now().strftime('%Y-%m-%d')

    # Google News URL for the keyword search and date range
    url = f"https://news.google.com/search?q={encoded_keyword}&hl=en-US&gl=US&ceid=US%3Aen&daterange={formatted_start_date}_{formatted_current_date}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the news articles on the page
    articles = soup.find_all('article')

    # Calculate the date threshold for comparison
    date_threshold = datetime.now() - timedelta(days=days)

    news_list = []
    for article in articles:
        # Extract the title, date published, and URL for each article
        title = article.find('h3', class_='ipQwMb ekueJc RD0gLb').text
        date_published = article.find('time')['datetime']
        article_url = 'https://news.google.com' + article.find('a')['href'][1:]

        # Convert the date string to a datetime object
        article_date = datetime.strptime(date_published, '%Y-%m-%dT%H:%M:%SZ')

        # Check if the article date is greater than or equal to the threshold
        if article_date >= date_threshold:
            # Append the article details to the news list
            news_list.append({
                'Title': title,
                'Date Published': date_published,
                'URL': article_url
            })

    return news_list


In [140]:
keyword = "Rep. Mike Levin"
days = 10

news_articles = get_news_articles(keyword, 3)


In [ ]:
news_articles

In [ ]:
pip install newscatcherapi

In [210]:
from newscatcherapi import NewsCatcherApiClient

newscatcherapi = NewsCatcherApiClient(x_api_key='biQVwhEMQQ1Vgt_2SkUajTemhW8H8LW3gJHuzSALBg8')

all_articles = newscatcherapi.get_search(q='Rep. Mike Levin',
                                         lang='en',
                                         countries='US',
                                         from_= '3 days ago',
                                         page_size=20)

In [ ]:
all_articles['articles']

In [209]:
import re
from datetime import datetime, timedelta
import pandas as pd
import time
import aiohttp
from newscatcherapi import NewsCatcherApiClient


def process_news_list(articles):
    news_data = []
    date_format = '%Y-%m-%d %H:%M:%S'

    for article in articles:
        name = ""
        title = article.get('title', "")
        url = article.get('link', "")
        date_str = article.get('published_date', "")
        summary = article.get('summary', "")

        # Parse and format the date
        if date_str:
            try:
                published_date = datetime.strptime(date_str, date_format)
                date = published_date.strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                date = ""

        news = {
            "Name": name,
            "Title": title,
            "URL": url,
            "Date": date,
            "Summary": summary
        }
        news_data.append(news)

    df = pd.DataFrame(news_data)
    return df


async def main(csv_filename, sheet_name, days):
    news_data = []
    date_threshold = datetime.utcnow() - timedelta(days=days)

    if sheet_name:
        representatives = pd.read_excel(csv_filename, sheet_name)['full_name'].tolist()
    else:
        representatives = pd.read_csv(csv_filename)['full_name'].tolist()

    async with aiohttp.ClientSession() as session:
        for rep in representatives:
            rep_name = "Rep. " + rep
            news = newscatcherapi.get_search(q=rep_name,
                                             lang='en',
                                             countries='US',
                                             from_='5 days ago',
                                             sort_by='date',
                                             page_size=20)
            print(rep_name)
            print(news)
            if news['status'] == "ok":
                news_df = process_news_list(news['articles'])
                news_data.append(news_df)
            time.sleep(1)

    news_list = pd.concat(news_data, ignore_index=True)
    return news_list

In [ ]:
process_news_list(all_articles['articles'])

In [211]:
if __name__ == '__main__':
# Run the main function
  csv_filename = '1.xlsx'  # Replace with the actual CSV file name
  sheet_name = 'Region5'
  days = 2
  result = await main(csv_filename, sheet_name, days)

Rep. Greg Casar
{'status': 'ok', 'total_hits': 12, 'page': 1, 'total_pages': 1, 'page_size': 12, 'articles': [{'title': '$2M Grant To Fund 58 Success Coaches At San Antonio Schools', 'author': '', 'published_date': '2023-07-05 10:55:00', 'published_date_precision': 'full', 'link': 'https://woai.iheart.com/featured/san-antonios-first-news/content/2023-07-05-2m-grant-to-fund-58-success-coaches-at-san-antonio-schools', 'clean_url': 'iheart.com', 'excerpt': 'More than $2 million federal dollars will fund the placement of volunteers at local schools to help motivate students.', 'summary': "More than $2 million federal dollars will fund the placement of volunteers at local schools to help motivate students. U.S. Reps. Joaquin Castro and Greg Casar recently announced that they'd secured the AmeriCorps funding for the San Antonio chapter of the nonprofit City Year. The funding will place 58 Student Success Coaches on the campuses of eight local schools to help kids improve their literacy, math

In [212]:
result

,Name,Title,URL,Date,Summary
0,,$2M Grant To Fund 58 Success Coaches At San An...,https://woai.iheart.com/featured/san-antonios-...,2023-07-05 10:55:00,More than $2 million federal dollars will fund...
1,,$2 million grant places 58 Student Success Coa...,https://www.ksat.com/news/local/2023/07/04/2-m...,2023-07-04 23:24:02,SAN ANTONIO\n – U.S. Reps. Joaquin Castro and ...
2,,Kremlin open to talks over potential prisoner ...,https://headtopics.com/us/kremlin-open-to-talk...,2023-07-04 12:16:00,The U.S. ambassador to Moscow was allowed to v...
3,,Olivia Dunne shares 'true tragedy' with her mi...,https://headtopics.com/us/olivia-dunne-shares-...,2023-07-04 08:55:00,Olivia Dunne posted a video to her ever-growin...
4,,Happy Fourth by Teresa Burns Parkhurst,https://headtopics.com/us/happy-fourth-by-tere...,2023-07-04 08:50:00,Happy Fourth by Teresa Burns Parkhurst Date: J...
...,...,...,...,...,...
61,,Congressional Republicans offer their own stud...,https://www.1310kfka.com/abc-news/politics/07/...,2023-07-04 11:05:31,Photo by Mike Kline (notkalvin)/Getty Images\n...
62,,Congressional Republicans offer their own stud...,https://www.wmay.com/2023/07/04/congressional-...,2023-07-04 11:05:31,Photo by Mike Kline (notkalvin)/Getty Images\n...
63,,Congressional Republicans offer their own stud...,https://abc7ny.com/congressional-republicans-o...,2023-07-03 23:15:00,In the wake of the Supreme Court's Friday ruli...
64,,Congressional Republicans offer their own stud...,https://abc11.com/congressional-republicans-of...,2023-07-03 22:45:44,In the wake of the Supreme Court's Friday ruli...


In [177]:
import re
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
from newscatcherapi import NewsCatcherApiClient

newscatcherapi = NewsCatcherApiClient(x_api_key='biQVwhEMQQ1Vgt_2SkUajTemhW8H8LW3gJHuzSALBg8')

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def process_news(info, date_threshold, rep):
    date_format = '%a, %d %b %Y %H:%M:%S %Z'
    news_row = None
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(info["url"]) as response:
                if response.status == 200:
                    content = await response.text()
                    if content and not content.startswith("https://consent.google.com/"):
                        final_url = extract_final_url(content)
                        if final_url:
                            published_date = datetime.strptime(info["published date"], date_format)
                            if published_date >= date_threshold:
                                news_row = {
                                    'Name': rep,
                                    'Title': info["title"],
                                    'Date': info["published date"],
                                    'URL': final_url
                                }
    except aiohttp.ClientError as e:
        print("URL not working:", e)

    return pd.DataFrame(news_row, index=[0]) if news_row else None

def extract_final_url(html_content):
    match = re.search(r'<a\s+href=[\'"]?([^\'" >]+)', html_content)
    if match:
        return match.group(1)
    return None

async def main(csv_filename, sheet_name, days):
    google_news = GNews()
    google_news.max_results = 10
    google_news.country = 'United States'
    google_news.language = 'english'
    google_news.exclude_websites = ["congress.gov", 'house.gov', "senate.gov"]
    news_list = pd.DataFrame(columns=['Name', 'Title', 'Date', 'URL'])
    date_threshold = datetime.utcnow() - timedelta(days=days)

    if sheet_name:
        representatives = pd.read_excel(csv_filename, sheet_name)['full_name'].tolist()
    else:
        representatives = pd.read_csv(csv_filename)['full_name'].tolist()

    tasks = []
    results = []  # Store the data frames returned by process_news
    async with aiohttp.ClientSession() as session:
        for rep in representatives:
            print(rep)
            rep_name = "Rep." + rep
            news = newscatcherapi.get_search(q='Rep. Mike Levin',
                                         lang='en',
                                         countries='US',
                                         from_= '3 days ago',
                                         page_size=20)

            print(news)
            for info in news:
                task = asyncio.create_task(process_news(info, date_threshold, rep))
                tasks.append(task)

                # Limit the number of simultaneous requests
                if len(tasks) >= 10:
                    # Await the tasks and store the data frames in the results list
                    news_data = await asyncio.gather(*tasks)
                    results.extend(news_data)
                    tasks = []

        if tasks:
            news_data = await asyncio.gather(*tasks)
            results.extend(news_data)

    if results:
        news_list = pd.concat(results, ignore_index=True)

    return news_list

In [178]:
if __name__ == '__main__':
# Run the main function
  csv_filename = '1.xlsx'  # Replace with the actual CSV file name
  sheet_name = 'Region5'
  days = 2
  result = await main(csv_filename, sheet_name, days)

/usr/local/lib/python3.10/dist-packages/openpyxl/descriptors/base.py:43: RuntimeWarning: coroutine 'main' was never awaited
  super(Typed, self).__set__(instance, value)
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1193' coro=<process_news() done, defined at <ipython-input-157-199b6b0ae730>:14> exception=TypeError('string indices must be integers')>
Traceback (most recent call last):
  File "<ipython-input-157-199b6b0ae730>", line 19, in process_news
    async with session.get(info["url"]) as response:
TypeError: string indices must be integers
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1194' coro=<process_news() done, defined at <ipython-input-157-199b6b0ae730>:14> exception=TypeError('string indices must be integers')>
Traceback (most recent call last):
  File "<ipython-input-157-199b6b0ae730>", line 19, in process_news
    async with session.get(info["url"]) as response:
TypeError: string indices must be i

Greg Casar


TypeError: ignored

In [ ]:
result

In [121]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Replace the path with the location of your service account key file
cred = credentials.Certificate('/content/key.json')

# Initialize the Firebase Admin SDK
firebase_admin.initialize_app(cred)

# Create a Firestore client
db = firestore.client()

In [ ]:
def upload_dataframe_to_firestore(dataframe, collection_name):
    # Convert the DataFrame to a list of dictionaries
    data = dataframe.to_dict(orient='records')

    # Upload each document in the list to Firestore
    for document in data:
        # Retrieve the URL and Name values from the document
        url = document.get('URL')
        name = document.get('Name')

        # Query Firestore to check if a document with the same URL and Name already exists
        query = db.collection(collection_name).where('URL', '==', url).where('Name', '==', name)
        existing_docs = query.get()

        if len(existing_docs) > 0:
            print(f"Skipping duplicate document with URL: {url} and Name: {name}")
            continue

        # Generate a new document ID
        doc_ref = db.collection(collection_name).document()

        # Set the document data
        doc_ref.set(document)

    print("DataFrame uploaded to Firestore successfully.")

# Example usage
collection_name = "news_collection"
upload_dataframe_to_firestore(result, collection_name)

In [122]:
def retrieve_data_by_field(collection_name, field_name, search_value):
    # Create a reference to the Firestore collection
    collection_ref = db.collection(collection_name)

    # Get all documents from the collection
    all_documents = collection_ref.get()

    # Process the documents and filter based on the search field and value
    for doc in all_documents:
        # Access the document data
        data = doc.to_dict()

        # Check if the search field and value match the document
        if field_name in data and data[field_name] == search_value:
            # Do something with the matching data
            print(data["Title"])


def retrieve_data_by_date(collection_name, search_date):
    # Create a reference to the Firestore collection
    collection_ref = db.collection(collection_name)

    # Get all documents from the collection
    all_documents = collection_ref.get()

    # Process the documents and filter based on the search date
    for doc in all_documents:
        # Access the document data
        data = doc.to_dict()

        # Get the "Date" field value and convert it to a datetime object
        date_str = data["Date"]
        date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %Z").date()

        # Check if the date matches the search date
        if date.strftime("%B %d") == search_date:
            # Do something with the matching data
            print(data["Title"])

In [ ]:
# Example usage
collection_name = "news_collection"
search_date = datetime.datetime(2023, 7, 4).date().strftime("%B %d")
retrieve_data_by_date(collection_name, search_date)


In [123]:
# Example usage
collection_name = "news_collection"
field_name = "Name"
search_value = "Mike Levin"
retrieve_data_by_field(collection_name, field_name, search_value)

Playbook: The political fireworks to watch today - POLITICO


# Old Code

In [ ]:
date_format = '%a, %d %b %Y %H:%M:%S %Z'
seven_days_ago = datetime.now() - timedelta(days=2)
df_final = pd.DataFrame(columns=['title', 'rep', 'link',"publisher"])
for ind in df.index:
    rep = ("Rep." + df['full_name'][ind])
    print(rep)
    news=google_news.get_news(rep)
    for info in news:
      try:
        url = requests.get(info["url"],timeout=5).url
        if datetime.strptime(info["published date"], date_format) >= seven_days_ago:
          if not ".gov" in url:
          #df_final.append([info["title"]],[rep],[url],[info["publisher"]])
            print(info["title"] + info["published date"])
            print(url)
        ##print(info["title"] + info["published date"] + " " + info["url"])
            new_row = pd.DataFrame({
                'Name': [df['full_name']],
                'Title': [info["title"]],
                'URL': [url]
            })

            # Concatenate the new row with the existing DataFrame
            news_list = pd.concat([news_list, new_row], ignore_index=True)
      except:
        print("url not working, try: ")
        print(info["url"])
    print("-"*100)


In [ ]:
date_format = '%a, %d %b %Y %H:%M:%S %Z'
seven_days_ago = datetime.now() - timedelta(days=7)
rep = ("Rep.Jerry L. Carl")
print(rep)
news=google_news.get_news(rep)
for info in news:
      try:
        response = requests.get(info["url"], allow_redirects=True)
        url = response.headers.get("Location") if "Location" in response.headers else response.url
        if datetime.strptime(info["published date"], date_format) >= seven_days_ago:
            if url and not url.startswith("https://consent.google.com/"):
                print(info["title"] + info["published date"])
                print(url)
                new_row = pd.DataFrame({
                    'Name': [rep],
                    'Title': [info["title"]],
                    'Date': [info["published date"]],
                    'URL': [url]
                })

                # Concatenate the new row with the existing DataFrame
                news_list = pd.concat([news_list, new_row], ignore_index=True)
      except:
        print("url not working")
print("-"*100)

## Get Article Text from URL Not as a full function
This is used when the publisher requires you to get subscription

In [ ]:
for site in urllist:
    url = "https://lexper.p.rapidapi.com/v1.1/extract"
    querystring = {"url":site,"js_timeout":"30","media":"true"}
    headers = {
	    "X-RapidAPI-Key": "8a62fe820amsh2a4a72771032a6cp1b6f99jsn339a1fcd31ca",
  	  "X-RapidAPI-Host": "lexper.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)

    print(site)
    html = (response.json()['article']['html'])
    htmlParse = bs(html, 'html.parser')
    str=""
    for para in htmlParse.find_all(['p','h1','h2','h3']):
      str = str + para.get_text().lstrip() + "\n"
    print(str)
    print("-"*100)


    sleep(1.5)